In [1]:
import pandas as pd

# Path to the Parquet file
parquet_file_path = 'dataset\\trans_data_TEST.parquet'

# Load the Parquet file into a pandas DataFrame
df = pd.read_parquet(parquet_file_path)

# Display the first few rows of the DataFrame
print(df.head())


   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         1  2022-06-01 00:25:41   2022-06-01 00:48:22              1.0   
1         1  2022-06-01 00:44:40   2022-06-01 01:01:48              1.0   
2         2  2022-06-01 00:23:07   2022-06-01 00:39:50              1.0   
3         1  2022-06-01 00:25:53   2022-06-01 00:57:06              2.0   
4         1  2022-06-01 00:23:58   2022-06-01 00:33:43              0.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0          11.00         1.0                  N            70            48   
1           4.20         1.0                  N           170           226   
2           9.49         1.0                  N           264           113   
3          12.10         1.0                  N           132            17   
4           1.80         1.0                  N           140           163   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  \


In [2]:
# Calculate mean and standard deviation
mean_fare = df['fare_amount'].mean()
std_fare = df['fare_amount'].std()

# Create a new column for the standardized fare amounts
df['fare_amount_sigmas'] = (df['fare_amount'] - mean_fare) / std_fare

# Filter out rows with 'fare_amount' beyond 3 standard deviations
df = df[(df['fare_amount_sigmas'] < 3) & (df['fare_amount_sigmas'] > -3)]

# Filter out rows with non-positive 'fare_amount'
df = df[df['fare_amount'] > 0]

# Drop the 'fare_amount_sigmas' column
df.drop("fare_amount_sigmas", inplace=True, axis=1)

# Display the first few rows of the DataFrame to verify the changes
print(df.head())

   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         1  2022-06-01 00:25:41   2022-06-01 00:48:22              1.0   
1         1  2022-06-01 00:44:40   2022-06-01 01:01:48              1.0   
2         2  2022-06-01 00:23:07   2022-06-01 00:39:50              1.0   
3         1  2022-06-01 00:25:53   2022-06-01 00:57:06              2.0   
4         1  2022-06-01 00:23:58   2022-06-01 00:33:43              0.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0          11.00         1.0                  N            70            48   
1           4.20         1.0                  N           170           226   
2           9.49         1.0                  N           264           113   
3          12.10         1.0                  N           132            17   
4           1.80         1.0                  N           140           163   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  \


In [3]:
# Ensure that 'tpep_dropoff_datetime' and 'tpep_pickup_datetime' are in datetime format
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])

# Add the new 'duration_in_minutes' column to the DataFrame
df["duration_in_minutes"] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).dt.total_seconds() / 60

# Extract datetime components and replace original datetime columns
df["pickup_year"] = df["tpep_pickup_datetime"].dt.year
df["pickup_day"] = df["tpep_pickup_datetime"].dt.day
df["pickup_day_of_week"] = df["tpep_pickup_datetime"].dt.dayofweek
df["pickup_hour"] = df["tpep_pickup_datetime"].dt.hour
df["pickup_minute"] = df["tpep_pickup_datetime"].dt.minute

# Drop the original datetime columns
df.drop("tpep_pickup_datetime", inplace=True, axis=1)
df.drop("tpep_dropoff_datetime", inplace=True, axis=1)

# Display the first few rows of the DataFrame to verify the changes
print(df.head())

   VendorID  passenger_count  trip_distance  RatecodeID store_and_fwd_flag  \
0         1              1.0          11.00         1.0                  N   
1         1              1.0           4.20         1.0                  N   
2         2              1.0           9.49         1.0                  N   
3         1              2.0          12.10         1.0                  N   
4         1              0.0           1.80         1.0                  N   

   PULocationID  DOLocationID  payment_type  fare_amount  extra  ...  \
0            70            48             1         32.0   3.00  ...   
1           170           226             1         14.0   3.00  ...   
2           264           113             1         26.0   0.50  ...   
3           132            17             2         37.0   1.75  ...   
4           140           163             1          9.0   3.00  ...   

   improvement_surcharge  total_amount  congestion_surcharge  airport_fee  \
0                    

In [4]:
import numpy as np


# Ensure that 'pickup_year' is extracted and present in the DataFrame
if 'pickup_year' not in df.columns:
    df['pickup_year'] = pd.to_datetime(df['tpep_pickup_datetime']).dt.year

# Remove rows that are not from 2022
df.loc[df["pickup_year"] != 2022, "pickup_year"] = np.nan

# Replace all rows with trip_distance > 60 because they are outliers
df.loc[df["trip_distance"] > 60, "trip_distance"] = np.nan

# Remove all rows with RatecodeID > 6 because according to docs RatecodeID can only go from 1 to 6
df.loc[df["RatecodeID"] > 6, "RatecodeID"] = np.nan

# Remove rows with PULocationID > 263 or DOLocationID > 263 because they don't add value to the model
df.loc[df["PULocationID"] > 263, "PULocationID"] = np.nan
df.loc[df["DOLocationID"] > 263, "DOLocationID"] = np.nan

# Filter out rows where total_amount is 400 or more
df = df[df['total_amount'] < 400]

# Display the first few rows of the DataFrame to verify the changes
print(df.head())

   VendorID  passenger_count  trip_distance  RatecodeID store_and_fwd_flag  \
0         1              1.0          11.00         1.0                  N   
1         1              1.0           4.20         1.0                  N   
2         2              1.0           9.49         1.0                  N   
3         1              2.0          12.10         1.0                  N   
4         1              0.0           1.80         1.0                  N   

   PULocationID  DOLocationID  payment_type  fare_amount  extra  ...  \
0          70.0          48.0             1         32.0   3.00  ...   
1         170.0         226.0             1         14.0   3.00  ...   
2           NaN         113.0             1         26.0   0.50  ...   
3         132.0          17.0             2         37.0   1.75  ...   
4         140.0         163.0             1          9.0   3.00  ...   

   improvement_surcharge  total_amount  congestion_surcharge  airport_fee  \
0                    

In [5]:
# Replace None values in the 'store_and_fwd_flag' column with 'N'
df["store_and_fwd_flag"].replace({None: "N"}, inplace=True)

# Display the first few rows of the DataFrame to verify the changes
print(df.head())


   VendorID  passenger_count  trip_distance  RatecodeID store_and_fwd_flag  \
0         1              1.0          11.00         1.0                  N   
1         1              1.0           4.20         1.0                  N   
2         2              1.0           9.49         1.0                  N   
3         1              2.0          12.10         1.0                  N   
4         1              0.0           1.80         1.0                  N   

   PULocationID  DOLocationID  payment_type  fare_amount  extra  ...  \
0          70.0          48.0             1         32.0   3.00  ...   
1         170.0         226.0             1         14.0   3.00  ...   
2           NaN         113.0             1         26.0   0.50  ...   
3         132.0          17.0             2         37.0   1.75  ...   
4         140.0         163.0             1          9.0   3.00  ...   

   improvement_surcharge  total_amount  congestion_surcharge  airport_fee  \
0                    

In [6]:
# Create copies of the DataFrame for training and testing
X_train = df.copy()
X_test = df.copy()

# Assign target columns to y_train
y_train_total_amount = X_train["total_amount"]
y_train_duration_in_minutes = X_train["duration_in_minutes"]

# Drop target columns from X_train
X_train.drop(["total_amount", "duration_in_minutes"], inplace=True, axis=1)

# Assign target columns to y_test
y_test_total_amount = X_test["total_amount"]
y_test_duration_in_minutes = X_test["duration_in_minutes"]

# Drop target columns from X_test
X_test.drop(["total_amount", "duration_in_minutes"], inplace=True, axis=1)

# Output the results
print(X_train.head())
print(y_train_total_amount.head())
print(y_train_duration_in_minutes.head())
print(X_test.head())
print(y_test_total_amount.head())
print(y_test_duration_in_minutes.head())


   VendorID  passenger_count  trip_distance  RatecodeID store_and_fwd_flag  \
0         1              1.0          11.00         1.0                  N   
1         1              1.0           4.20         1.0                  N   
2         2              1.0           9.49         1.0                  N   
3         1              2.0          12.10         1.0                  N   
4         1              0.0           1.80         1.0                  N   

   PULocationID  DOLocationID  payment_type  fare_amount  extra  ...  \
0          70.0          48.0             1         32.0   3.00  ...   
1         170.0         226.0             1         14.0   3.00  ...   
2           NaN         113.0             1         26.0   0.50  ...   
3         132.0          17.0             2         37.0   1.75  ...   
4         140.0         163.0             1          9.0   3.00  ...   

   tip_amount  tolls_amount  improvement_surcharge  congestion_surcharge  \
0        2.00         

In [16]:
X_train.columns

Index(['trip_distance', 'PULocationID', 'DOLocationID', 'payment_type',
       'fare_amount', 'mta_tax', 'tolls_amount', 'airport_fee', 'pickup_year',
       'pickup_day', 'pickup_day_of_week', 'pickup_hour', 'pickup_minute'],
      dtype='object')

In [15]:
import pandas as pd

def remove_columns_in_place(dfs, columns_to_remove):
    """
    Remove specified columns from a list of DataFrames in place.

    Parameters:
    dfs (list of pd.DataFrame): A list of DataFrames to modify.
    columns_to_remove (list): A list of column names to remove from each DataFrame.
    """
    for i in range(len(dfs)):
        dfs[i].drop(columns=columns_to_remove, axis=1, inplace=True)

# List of columns to remove
columns_to_remove = ['improvement_surcharge',  'congestion_surcharge', 'VendorID', 'passenger_count', 'RatecodeID', 'store_and_fwd_flag', 'extra', 'tip_amount']

# Your DataFrames
df_list = [X_train]

# Remove columns from all DataFrames in place
remove_columns_in_place(df_list, columns_to_remove)

# Display the first few rows of the cleaned DataFrame to verify the changes
print(X_train.columns)


KeyError: "['improvement_surcharge', 'congestion_surcharge', 'VendorID', 'passenger_count', 'RatecodeID', 'store_and_fwd_flag', 'extra', 'tip_amount'] not found in axis"

In [34]:
X_train.head(10)

,trip_distance,PULocationID,DOLocationID,payment_type,mta_tax,tolls_amount,airport_fee,pickup_year,pickup_day,pickup_day_of_week,pickup_hour,pickup_minute
0,11.00,70.0,48.0,1,0.5,6.55,0.00,2022.0,1,2,0,25
1,4.20,170.0,226.0,1,0.5,0.00,0.00,2022.0,1,2,0,44
2,9.49,NaN,113.0,1,0.5,6.55,1.25,2022.0,1,2,0,23
3,12.10,132.0,17.0,2,0.5,0.00,1.25,2022.0,1,2,0,25
4,1.80,140.0,163.0,1,0.5,0.00,0.00,2022.0,1,2,0,23
5,2.02,148.0,158.0,1,0.5,0.00,0.00,2022.0,1,2,0,1
6,8.08,158.0,116.0,1,0.5,0.00,0.00,2022.0,1,2,0,16
7,4.30,246.0,262.0,1,0.5,0.00,0.00,2022.0,1,2,0,11
8,8.78,197.0,191.0,1,0.5,0.00,0.00,2022.0,1,2,0,21
9,1.76,48.0,186.0,1,0.5,0.00,0.00,2022.0,1,2,0,23


In [20]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score

# Create the LightGBM dataset
train_data = lgb.Dataset(X_train)
test_data = lgb.Dataset(y_test_total_amount)

# Set parameters for LightGBM
params = {
    'objective': 'regression', # Set the objective for regression problem
    'metric': 'mse', # Use mean squared error as evaluation metric
    'num_leaves': 31, # Maximum number of leaves in one tree
    'learning_rate': 0.05, # Learning rate
    'feature_fraction': 0.9, # Feature fraction (randomly select part of features on each iteration)
    'bagging_fraction': 0.8, # Bagging fraction (randomly select part of data without resampling)
    'bagging_freq': 5, # Frequency for bagging
    'verbose': 0 # Verbose level
}

# Train the model
num_round = 100 # Number of boosting rounds
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

# Make predictions
y_pred = bst.predict(X_test, num_iteration=bst.best_iteration)

# Evaluate the model
mse = mean_squared_error(y_test_total_amount, y_pred)
print('Mean Squared Error:', mse)

# Calculate R-squared
r2 = r2_score(y_test_total_amount, y_pred)
print(f"R-squared: {r2:.2f}")

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped t

ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: store_and_fwd_flag: object

In [30]:
import pandas as pd

# Function to remove specified columns from a DataFrame
def remove_columns_in_place(dfs, columns_to_remove):
    """
    Remove specified columns from a list of DataFrames in place.

    Parameters:
    dfs (list of pd.DataFrame): A list of DataFrames to modify.
    columns_to_remove (list): A list of column names to remove from each DataFrame.
    """
    for df in dfs:
        columns_to_remove_existing = [col for col in columns_to_remove if col in df.columns]
        df.drop(columns=columns_to_remove_existing, axis=1, inplace=True)

# List of columns to remove
columns_to_remove = ['improvement_surcharge', 'congestion_surcharge', 'VendorID', 
                     'passenger_count', 'RatecodeID', 'store_and_fwd_flag', 'extra', 'tip_amount', 'fare_amount']

# Your DataFrames
df_list = [X_train, X_test]

# Remove columns from all DataFrames in place
remove_columns_in_place(df_list, columns_to_remove)

# Display the first few rows of the cleaned DataFrame to verify the changes
print(X_train.columns)

Index(['trip_distance', 'PULocationID', 'DOLocationID', 'payment_type',
       'mta_tax', 'tolls_amount', 'airport_fee', 'pickup_year', 'pickup_day',
       'pickup_day_of_week', 'pickup_hour', 'pickup_minute'],
      dtype='object')


In [31]:


# Example LightGBM training code
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import pickle
# Prepare data for LightGBM
train_data = lgb.Dataset(X_train, label=y_train_total_amount)
test_data = lgb.Dataset(X_test, label=y_test_total_amount, reference=train_data)

# Set parameters for LightGBM
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

# Train the model
num_round = 100
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])
# Save the model to a pickle file
with open('lgb_model.pickle', 'wb') as model_file:
    pickle.dump(bst, model_file)

# Make predictions
y_pred = bst.predict(X_test, num_iteration=bst.best_iteration)

# Evaluate the model
mse = mean_squared_error(y_test_total_amount, y_pred)
print(f'Mean Squared Error: {mse}')
r2 = r2_score(y_test_total_amount, y_pred)
print(f"R-squared: {r2:.2f}")



[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.077288 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 862
[LightGBM] [Info] Number of data points in the train set: 3533786, number of used features: 12
[LightGBM] [Info] Start training from score 22.143934
Mean Squared Error: 29.45662205205747
R-squared: 0.90


In [36]:
import pandas as pd
import pickle

import pandas as pd
import pickle

def load_and_predict_single(model_filename, single_instance):
    # Load the model from the pickle file
    with open(model_filename, 'rb') as model_file:
        bst = pickle.load(model_file)

    # Convert the single instance to a DataFrame with one row if it's a dictionary
    if isinstance(single_instance, dict):
        single_instance = pd.DataFrame([single_instance])
    elif isinstance(single_instance, pd.Series):
        single_instance = single_instance.to_frame().T

    # Make prediction using the loaded model
    y_pred = bst.predict(single_instance, num_iteration=bst.best_iteration)

    return y_pred[0]

# Example usage for a single test instance
single_instance_data = {
    'trip_distance': 12.10,
    'PULocationID': 132.0,
    'DOLocationID': 17.0,
    'payment_type': 2,
    'mta_tax': 0.5,
    'tolls_amount': 0,
    'airport_fee': 0,
    'pickup_year': 2022,
    'pickup_day': 1,
    'pickup_day_of_week': 2,
    'pickup_hour': 0,
    'pickup_minute': 23
}

predicted_value = load_and_predict_single('lgb_model.pickle', single_instance_data)
print(f'Predicted Value for the single instance: {predicted_value}')


Predicted Value for the single instance: 38.26224186462784


 37.0

In [32]:
X_train.head(10)

,trip_distance,PULocationID,DOLocationID,payment_type,mta_tax,tolls_amount,airport_fee,pickup_year,pickup_day,pickup_day_of_week,pickup_hour,pickup_minute
0,11.00,70.0,48.0,1,0.5,6.55,0.00,2022.0,1,2,0,25
1,4.20,170.0,226.0,1,0.5,0.00,0.00,2022.0,1,2,0,44
2,9.49,NaN,113.0,1,0.5,6.55,1.25,2022.0,1,2,0,23
3,12.10,132.0,17.0,2,0.5,0.00,1.25,2022.0,1,2,0,25
4,1.80,140.0,163.0,1,0.5,0.00,0.00,2022.0,1,2,0,23
5,2.02,148.0,158.0,1,0.5,0.00,0.00,2022.0,1,2,0,1
6,8.08,158.0,116.0,1,0.5,0.00,0.00,2022.0,1,2,0,16
7,4.30,246.0,262.0,1,0.5,0.00,0.00,2022.0,1,2,0,11
8,8.78,197.0,191.0,1,0.5,0.00,0.00,2022.0,1,2,0,21
9,1.76,48.0,186.0,1,0.5,0.00,0.00,2022.0,1,2,0,23


In [35]:
single_instance_data = {
    'trip_distance': 12.10,
    'PULocationID': 132.0,
    'DOLocationID': 17.0,
    'payment_type': 2,
    'mta_tax': 0.5,
    'tolls_amount': 0,
    'airport_fee': 0,
    'pickup_year': 2022,
    'pickup_day': 1,
    'pickup_day_of_week': 2,
    'pickup_hour': 0,
    'pickup_minute': 23
}



In [ ]:
          17.0             2         37.0   1.75

VendorID  passenger_count  trip_distance  RatecodeID store_and_fwd_flag  \
0         1              1.0          11.00         1.0                  N   
1         1              1.0           4.20         1.0                  N   
2         2              1.0           9.49         1.0                  N   
3         1              2.0          12.10         1.0                  N   
4         1              0.0           1.80         1.0                  N   

   PULocationID  DOLocationID  payment_type  fare_amount  extra  ...  \
0          70.0          48.0             1         32.0   3.00  ...   
1         170.0         226.0             1         14.0   3.00  ...   
2           NaN         113.0             1         26.0   0.50  ...   
3         132.0          17.0             2         37.0   1.75  ...   
4         140.0         163.0             1          9.0   3.00  ...   

   improvement_surcharge  total_amount  congestion_surcharge  airport_fee  \
0                    0.3         44.35                   2.5         0.00   
1                    0.3         17.80                   2.5         0.00   
2                    0.3         42.60                   2.5         1.25   
3                    0.3         39.55                   0.0         1.25   
4                    0.3         15.35                   2.5         0.00   

   duration_in_minutes  pickup_year  pickup_day  pickup_day_of_week  \
0            22.683333       2022.0           1                   2   
1            17.133333       2022.0           1                   2   
2            16.716667       2022.0           1                   2   
...
3            0             25  
4            0             23  